# Predicting immediate approval of claims

This project aims to build a predictive model that can predict the probability that a particular claim will be approved immediately or not by the insurance company.

See the [README.md](https://github.com/cpatrickalves/kaggle-insurance-claim-classification) file and [competitions' page](https://www.kaggle.com/c/competicao-dsa-machine-learning-dec-2019/overview) for further details.

#### Loading the datasets

In [1]:
# Loading useful Python packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# loading datasets
train_df = pd.read_csv('data/dataset_treino.csv')
test_df = pd.read_csv('data/dataset_teste.csv')

In [8]:
train_df.head()

,ID,target,v1,v2,v3,v4,v5,v6,v7,v8,...,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,3,1,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,0.012941,...,8.000000,1.989780,0.035754,AU,1.804126,3.113719,2.024285,0,0.636365,2.857144
1,4,1,NaN,NaN,C,NaN,9.191265,NaN,NaN,2.301630,...,NaN,NaN,0.598896,AF,NaN,NaN,1.957825,0,NaN,NaN
2,5,1,0.943877,5.310079,C,4.410969,5.326159,3.979592,3.928571,0.019645,...,9.333333,2.477596,0.013452,AE,1.773709,3.922193,1.120468,2,0.883118,1.176472
3,6,1,0.797415,8.304757,C,4.225930,11.627438,2.097700,1.987549,0.171947,...,7.018256,1.812795,0.002267,CJ,1.415230,2.954381,1.990847,1,1.677108,1.034483
4,8,1,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Z,NaN,NaN,NaN,0,NaN,NaN


### Data Cleaning

The first step before perform any kind of statistical analysis and modeling is to clean the data.

Let's see the type of data we have.

In [10]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114321 entries, 0 to 114320
Columns: 133 entries, ID to v131
dtypes: float64(108), int64(6), object(19)
memory usage: 116.0+ MB


From the above, we can see that this data set has 114321 rows and 133 columns. 

Also, we have **114 numerical features** (columns) and **19 categorical features**.

Let's see if we have null values (also know as _NaN_)

In [13]:
# There are null values?
train_df.isnull().values.any()

True

In [21]:
# Null values amount for each column
train_df.isnull().sum().sort_values(ascending=False)

v30       60110
v113      55304
v102      51316
v85       50682
v119      50680
v51       50678
v123      50678
v23       50675
v78       49895
v115      49895
v69       49895
v131      49895
v16       49895
v122      49851
v80       49851
v9        49851
v37       49843
v118      49843
v130      49843
v19       49843
v92       49843
v95       49843
v97       49843
v20       49840
v65       49840
v121      49840
v11       49836
v39       49836
v73       49836
v90       49836
          ...  
v3         3457
v31        3457
v21         611
v22         500
v112        382
v34         111
v40         111
v12          86
v50          86
v10          84
v125         77
v114         30
v14           4
v52           3
v91           3
v107          3
v24           0
v38           0
v47           0
v62           0
v66           0
v129          0
v71           0
v72           0
v74           0
v75           0
v79           0
v110          0
target        0
ID            0
Length: 133, dtype: int6

So, we have a lot of null values in several columns.

Let's check the percentage of null values for each column.

In [28]:
null_values = train_df.isnull().sum()
null_values = round((null_values/train_df.shape[0] * 100), 2)
null_values.sort_values(ascending=False)

v30       52.58
v113      48.38
v102      44.89
v51       44.33
v85       44.33
v23       44.33
v123      44.33
v119      44.33
v115      43.64
v78       43.64
v69       43.64
v131      43.64
v16       43.64
v122      43.61
v80       43.61
v9        43.61
v37       43.60
v130      43.60
v20       43.60
v19       43.60
v92       43.60
v95       43.60
v97       43.60
v65       43.60
v118      43.60
v121      43.60
v53       43.59
v42       43.59
v68       43.59
v67       43.59
          ...  
v3         3.02
v31        3.02
v21        0.53
v22        0.44
v112       0.33
v40        0.10
v34        0.10
v12        0.08
v50        0.08
v125       0.07
v10        0.07
v114       0.03
v129       0.00
target     0.00
v107       0.00
v14        0.00
v24        0.00
v38        0.00
v47        0.00
v52        0.00
v62        0.00
v66        0.00
v71        0.00
v72        0.00
v74        0.00
v75        0.00
v79        0.00
v91        0.00
v110       0.00
ID         0.00
Length: 133, dtype: floa

Considering that we are dealing with anonymous data and we can't know the meaning of the data, I'll remove all columns with more than 40% of null values.

In [36]:
# Get the names of the columns that have more than 40% of null values
high_nan_rate_columns = null_values[null_values > 40].index

# Make a copy if the original datasets and remove the columns
train_df_cleaned = train_df.copy()
test_df_cleaned = test_df.copy()
train_df_cleaned.drop(high_nan_rate_columns, axis=1, inplace=True)
test_df_cleaned.drop(high_nan_rate_columns, axis=1, inplace=True)

# Remove the ID column (it is now useful for modeling)
train_df_cleaned.drop(['ID'], axis=1, inplace=True)
test_df_cleaned.drop(['ID'], axis=1, inplace=True)

train_df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114321 entries, 0 to 114320
Data columns (total 30 columns):
target    114321 non-null int64
v3        110864 non-null object
v10       114237 non-null float64
v12       114235 non-null float64
v14       114317 non-null float64
v21       113710 non-null float64
v22       113821 non-null object
v24       114321 non-null object
v31       110864 non-null object
v34       114210 non-null float64
v38       114321 non-null int64
v40       114210 non-null float64
v47       114321 non-null object
v50       114235 non-null float64
v52       114318 non-null object
v56       107439 non-null object
v62       114321 non-null int64
v66       114321 non-null object
v71       114321 non-null object
v72       114321 non-null int64
v74       114321 non-null object
v75       114321 non-null object
v79       114321 non-null object
v91       114318 non-null object
v107      114318 non-null object
v110      114321 non-null object
v112      113939 non-null ob

Now we have only 30 columns in the data set.

But we still have null values that need to be handled.

In [38]:
null_values_columns = train_df_cleaned.isnull().sum().sort_values(ascending=False)
null_values_columns = null_values_columns[null_values_columns > 0]
null_values_columns

v56     6882
v31     3457
v3      3457
v21      611
v22      500
v112     382
v40      111
v34      111
v50       86
v12       86
v10       84
v125      77
v114      30
v14        4
v91        3
v107       3
v52        3
dtype: int64

In [40]:
train_df_cleaned[null_values_columns.index].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114321 entries, 0 to 114320
Data columns (total 17 columns):
v56     107439 non-null object
v31     110864 non-null object
v3      110864 non-null object
v21     113710 non-null float64
v22     113821 non-null object
v112    113939 non-null object
v40     114210 non-null float64
v34     114210 non-null float64
v50     114235 non-null float64
v12     114235 non-null float64
v10     114237 non-null float64
v125    114244 non-null object
v114    114291 non-null float64
v14     114317 non-null float64
v91     114318 non-null object
v107    114318 non-null object
v52     114318 non-null object
dtypes: float64(8), object(9)
memory usage: 14.8+ MB


From the above, there are 8 numeric columns and 9 categorical columns with nulls values.

For now, we will replace the null values by the MEAN value for each numeric column and for the MODE for each of the categorical columns.

In [49]:
##### Numerical columns
numerical_col_null_values = train_df_cleaned[null_values_columns.index].select_dtypes(include=['float64']).columns
# for each column
for c in numerical_col_null_values:
    # Get the mean
    mean = train_df_cleaned[c].mean()
    # replace the NaN by mode
    train_df_cleaned[c].fillna(mean, inplace=True)

##### Categorical columns
categ_col_null_values = train_df_cleaned[null_values_columns.index].select_dtypes(include=['object']).columns
# for each column
for c in categ_col_null_values:
    # Get the most frequent value (mode)
    mode = train_df_cleaned[c].value_counts().index[0]
    # replace the NaN by mode
    train_df_cleaned[c].fillna(mode, inplace=True)

In [50]:
# There are null values?
train_df_cleaned.isnull().values.any()

False